# Task 1: Data Exploration and Enrichment

## 1) Load CSVs and Preview Schema
Load the raw dataset and reference codes, then preview the schema.

In [3]:
from pathlib import Path
import pandas as pd

# Find repo root by walking up until a data/raw folder is found
cwd = Path.cwd().resolve()
root = None
for parent in [cwd] + list(cwd.parents):
    if (parent / "data" / "raw" / "ethiopia_fi_unified_data.csv").exists():
        root = parent
        break

if root is None:
    raise FileNotFoundError("Could not locate data/raw/ethiopia_fi_unified_data.csv from current working directory.")

raw_path = root / "data" / "raw" / "ethiopia_fi_unified_data.csv"
ref_path = root / "data" / "raw" / "reference_codes.csv"

df = pd.read_csv(raw_path)
ref = pd.read_csv(ref_path)

print("Raw data shape:", df.shape)
print("Reference codes shape:", ref.shape)

df.head()


Raw data shape: (43, 34)
Reference codes shape: (71, 4)


,record_id,record_type,category,pillar,indicator,indicator_code,indicator_direction,value_numeric,value_text,value_type,...,impact_direction,impact_magnitude,impact_estimate,lag_months,evidence_basis,comparable_country,collected_by,collection_date,original_text,notes
0,REC_0001,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,22.0,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,Baseline year,NaN
1,REC_0002,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,35.0,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,NaN,NaN
2,REC_0003,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,46.0,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,NaN,NaN
3,REC_0004,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,56.0,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,Gender disaggregated,NaN
4,REC_0005,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,36.0,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,Gender disaggregated,NaN


## 2) Basic EDA: Shapes, Ranges, and Value Counts
Quick distributions and temporal coverage checks.

In [4]:
df_info = {
    "shape": df.shape,
    "record_type_counts": df["record_type"].value_counts(),
    "pillar_counts": df["pillar"].value_counts(dropna=False),
    "indicator_code_counts": df["indicator_code"].value_counts().head(10),
    "source_type_counts": df["source_type"].value_counts(),
}

for k, v in df_info.items():
    print(f"\n{k}:")
    print(v)

# Temporal range
obs_dates = pd.to_datetime(df["observation_date"], errors="coerce")
print("\nObservation date range:", obs_dates.min(), "to", obs_dates.max())

# Events list
events = df[df["record_type"] == "event"]["indicator"].dropna().unique()
print("\nEvents:")
for evt in events:
    print("-", evt)



shape:
(43, 34)

record_type_counts:
record_type
observation    30
event          10
target          3
Name: count, dtype: int64

pillar_counts:
pillar
ACCESS           16
USAGE            11
NaN              10
GENDER            5
AFFORDABILITY     1
Name: count, dtype: int64

indicator_code_counts:
indicator_code
ACC_OWNERSHIP     7
ACC_FAYDA         4
ACC_MM_ACCOUNT    2
ACC_4G_COV        2
USG_P2P_COUNT     2
GEN_GAP_ACC       2
GEN_MM_SHARE      2
ACC_MOBILE_PEN    1
USG_P2P_VALUE     1
USG_ATM_COUNT     1
Name: count, dtype: int64

source_type_counts:
source_type
operator      15
survey        10
regulator      7
research       4
policy         3
calculated     2
news           2
Name: count, dtype: int64

Observation date range: 2014-12-31 00:00:00 to 2030-12-31 00:00:00

Events:
- Telebirr Launch
- Safaricom Ethiopia Commercial Launch
- M-Pesa Ethiopia Launch
- Fayda Digital ID Program Rollout
- Foreign Exchange Liberalization
- P2P Transaction Count Surpasses ATM
- M-Pesa Eth

## 3) Prepare Enrichment Rows (Observations, Events, Impact Links)
Create new rows matching the dataset schema.

In [5]:
base = {col: "" for col in df.columns}

def make_row(**kwargs):
    row = base.copy()
    row.update(kwargs)
    return row

new_rows = [
    make_row(
        record_id="REC_0034",
        record_type="observation",
        pillar="ACCESS",
        indicator="Account Ownership Rate",
        indicator_code="ACC_OWNERSHIP",
        indicator_direction="higher_better",
        value_numeric=49,
        value_type="percentage",
        unit="%",
        observation_date="2024-12-31",
        fiscal_year="2024",
        gender="all",
        location="national",
        source_name="World Bank Global Findex 2025",
        source_type="survey",
        source_url="https://data.worldbank.org/",
        confidence="high",
        collected_by="Faysal",
        collection_date="2026-02-01",
        original_text="Account ownership 49% in 2024 (Findex 2025)",
        notes="Confirms slow +3pp growth 2021–2024 despite supply boom; baseline for forecasts."
    ),
    make_row(
        record_id="REC_0035",
        record_type="observation",
        pillar="ACCESS",
        indicator="Account Ownership Rate",
        indicator_code="ACC_OWNERSHIP",
        indicator_direction="higher_better",
        value_numeric=57,
        value_type="percentage",
        unit="%",
        observation_date="2024-12-31",
        fiscal_year="2024",
        gender="male",
        location="national",
        source_name="World Bank Global Findex 2025",
        source_type="survey",
        source_url="https://birrmetrics.com/49-of-ethiopians-are-banked-as-findex-2025-highlights-the-next-inclusion-challenge",
        confidence="high",
        collected_by="Faysal",
        collection_date="2026-02-01",
        original_text="Men 57% account ownership (Findex 2025)",
        notes="Male account ownership; persistent gender gap."
    ),
    make_row(
        record_id="REC_0036",
        record_type="observation",
        pillar="ACCESS",
        indicator="Account Ownership Rate",
        indicator_code="ACC_OWNERSHIP",
        indicator_direction="higher_better",
        value_numeric=42,
        value_type="percentage",
        unit="%",
        observation_date="2024-12-31",
        fiscal_year="2024",
        gender="female",
        location="national",
        source_name="World Bank Global Findex 2025",
        source_type="survey",
        source_url="https://birrmetrics.com/49-of-ethiopians-are-banked-as-findex-2025-highlights-the-next-inclusion-challenge",
        confidence="high",
        collected_by="Faysal",
        collection_date="2026-02-01",
        original_text="Women 42% account ownership (Findex 2025)",
        notes="Female account ownership; persistent gender gap."
    ),
    make_row(
        record_id="REC_0037",
        record_type="observation",
        pillar="ACCESS",
        indicator="Account Ownership Rate",
        indicator_code="ACC_OWNERSHIP",
        indicator_direction="higher_better",
        value_numeric=73,
        value_type="percentage",
        unit="%",
        observation_date="2024-12-31",
        fiscal_year="2024",
        gender="all",
        location="urban",
        source_name="World Bank Global Findex 2025",
        source_type="survey",
        source_url="https://birrmetrics.com/49-of-ethiopians-are-banked-as-findex-2025-highlights-the-next-inclusion-challenge",
        confidence="high",
        collected_by="Faysal",
        collection_date="2026-02-01",
        original_text="Urban account ownership 73% in 2024",
        notes="Urban-rural gap remains wide; supports spatial analysis."
    ),
    make_row(
        record_id="REC_0038",
        record_type="observation",
        pillar="ACCESS",
        indicator="Account Ownership Rate",
        indicator_code="ACC_OWNERSHIP",
        indicator_direction="higher_better",
        value_numeric=43,
        value_type="percentage",
        unit="%",
        observation_date="2024-12-31",
        fiscal_year="2024",
        gender="all",
        location="rural",
        source_name="World Bank Global Findex 2025",
        source_type="survey",
        source_url="https://birrmetrics.com/49-of-ethiopians-are-banked-as-findex-2025-highlights-the-next-inclusion-challenge",
        confidence="high",
        collected_by="Faysal",
        collection_date="2026-02-01",
        original_text="Rural account ownership 43% in 2024",
        notes="Rural access lags urban by 30pp."
    ),
    make_row(
        record_id="REC_0039",
        record_type="observation",
        pillar="ACCESS",
        indicator="Account Ownership Rate",
        indicator_code="ACC_OWNERSHIP",
        indicator_direction="higher_better",
        value_numeric=53,
        value_type="percentage",
        unit="%",
        observation_date="2024-12-31",
        fiscal_year="2024",
        gender="all",
        location="income_richest_60",
        source_name="World Bank Global Findex 2025",
        source_type="survey",
        source_url="https://birrmetrics.com/49-of-ethiopians-are-banked-as-findex-2025-highlights-the-next-inclusion-challenge",
        confidence="high",
        collected_by="Faysal",
        collection_date="2026-02-01",
        original_text="Richest 60% account ownership 53% in 2024",
        notes="Income split supports distribution analysis."
    ),
    make_row(
        record_id="REC_0040",
        record_type="observation",
        pillar="ACCESS",
        indicator="Account Ownership Rate",
        indicator_code="ACC_OWNERSHIP",
        indicator_direction="higher_better",
        value_numeric=43,
        value_type="percentage",
        unit="%",
        observation_date="2024-12-31",
        fiscal_year="2024",
        gender="all",
        location="income_poorest_40",
        source_name="World Bank Global Findex 2025",
        source_type="survey",
        source_url="https://birrmetrics.com/49-of-ethiopians-are-banked-as-findex-2025-highlights-the-next-inclusion-challenge",
        confidence="high",
        collected_by="Faysal",
        collection_date="2026-02-01",
        original_text="Poorest 40% account ownership 43% in 2024",
        notes="Income split supports distribution analysis."
    ),
    make_row(
        record_id="REC_0041",
        record_type="observation",
        pillar="ACCESS",
        indicator="Mobile Money Registered Accounts",
        indicator_code="SUP_MM_REGISTERED_ACCOUNTS",
        indicator_direction="higher_better",
        value_numeric=136000000,
        value_type="count",
        unit="accounts",
        observation_date="2025-12-31",
        fiscal_year="2025",
        gender="all",
        location="national",
        source_name="NBE (via Birr Metrics)",
        source_type="regulator",
        source_url="https://birrmetrics.com/ethiopias-mobile-money-users-reach-136-million-digital-transactions-top-96-trillion-birr",
        confidence="high",
        collected_by="Faysal",
        collection_date="2026-02-01",
        original_text="Mobile money registered accounts reach 136 million",
        notes="Registration surge highlights inactive accounts; leading indicator for usage activation."
    ),
    make_row(
        record_id="REC_0042",
        record_type="observation",
        pillar="USAGE",
        indicator="Telebirr Registered Users",
        indicator_code="USG_TELEBIRR_USERS",
        indicator_direction="higher_better",
        value_numeric=58600000,
        value_type="count",
        unit="users",
        observation_date="2025-12-31",
        fiscal_year="2025",
        gender="all",
        location="national",
        source_name="Ethio Telecom / Stockmarket.et",
        source_type="operator",
        source_url="https://stockmarket.et/",
        confidence="high",
        collected_by="Faysal",
        collection_date="2026-02-01",
        original_text="Telebirr users reach 58.6 million in late 2025",
        notes="Updated user base for late 2025; supports usage growth tracking."
    ),
    make_row(
        record_id="REC_0043",
        record_type="observation",
        pillar="ACCESS",
        indicator="CICO / Agent Network",
        indicator_code="SUP_CICO_AGENTS",
        indicator_direction="higher_better",
        value_numeric=500000,
        value_type="count",
        unit="agents",
        observation_date="2025-06-30",
        fiscal_year="2025",
        gender="all",
        location="national",
        source_name="National Bank of Ethiopia",
        source_type="regulator",
        source_url="https://nbe.gov.et/wp-content/uploads/2025/07/CICO-Agent-Mapping-Strategy_NBE.pdf",
        confidence="high",
        collected_by="Faysal",
        collection_date="2026-02-01",
        original_text="Registered CICO agents about 500000 in mid 2025",
        notes="Agent network growth enables rural cash in cash out coverage."
    ),
    make_row(
        record_id="REC_0044",
        record_type="observation",
        pillar="ACCESS",
        indicator="Internet Penetration",
        indicator_code="ACC_INTERNET_PEN",
        indicator_direction="higher_better",
        value_numeric=21.7,
        value_type="percentage",
        unit="%",
        observation_date="2025-12-31",
        fiscal_year="2025",
        gender="all",
        location="national",
        source_name="DataReportal Digital 2026",
        source_type="research",
        source_url="https://datareportal.com/reports/digital-2026-ethiopia",
        confidence="high",
        collected_by="Faysal",
        collection_date="2026-02-01",
        original_text="Internet penetration at 21.7% of population end 2025",
        notes="Low but rising connectivity; proxy for app based payments readiness."
    ),
    make_row(
        record_id="EVT_0011",
        record_type="event",
        category="policy",
        indicator="National Digital Payments Strategy 2026-2030",
        indicator_code="EVT_NDPS_2026_2030",
        value_text="Launched",
        value_type="categorical",
        observation_date="2025-12-01",
        fiscal_year="2025",
        gender="all",
        location="national",
        source_name="National Bank of Ethiopia",
        source_type="regulator",
        source_url="https://nbe.gov.et/wp-content/uploads/2025/12/Ethiopia_NDPS_Draft_F.pdf",
        confidence="high",
        collected_by="Faysal",
        collection_date="2026-02-01",
        original_text="NDPS 2026-2030 draft launched in Dec 2025",
        notes="Policy focus on interoperability and active usage; likely boosts usage pillar."
    ),
    make_row(
        record_id="IMP_0001",
        record_type="impact_link",
        category="policy",
        pillar="USAGE",
        indicator="NDPS 2026-2030 Impact",
        indicator_code="EVT_NDPS_2026_2030",
        observation_date="2025-12-01",
        fiscal_year="2025",
        gender="all",
        location="national",
        source_name="National Bank of Ethiopia",
        source_type="regulator",
        source_url="https://nbe.gov.et/wp-content/uploads/2025/12/Ethiopia_NDPS_Draft_F.pdf",
        confidence="medium",
        related_indicator="USG_DIGITAL_PAYMENT",
        relationship_type="policy_impact",
        impact_direction="positive",
        impact_magnitude="medium",
        impact_estimate="+5-15pp",
        lag_months="18",
        evidence_basis="Comparable policy impacts in Kenya and Rwanda; strategy aims to activate dormant accounts.",
        comparable_country="Kenya and Rwanda",
        collected_by="Faysal",
        collection_date="2026-02-01",
        notes="Linked to EVT_0011."
    ),
    make_row(
        record_id="IMP_0002",
        record_type="impact_link",
        category="policy",
        pillar="USAGE",
        indicator="NDPS 2026-2030 Impact on Active Use",
        indicator_code="EVT_NDPS_2026_2030",
        observation_date="2025-12-01",
        fiscal_year="2025",
        gender="all",
        location="national",
        source_name="National Bank of Ethiopia",
        source_type="regulator",
        source_url="https://nbe.gov.et/wp-content/uploads/2025/12/Ethiopia_NDPS_Draft_F.pdf",
        confidence="medium",
        related_indicator="USG_ACTIVE_RATE",
        relationship_type="policy_impact",
        impact_direction="positive",
        impact_magnitude="medium",
        impact_estimate="+3-10pp",
        lag_months="24",
        evidence_basis="Policy focus on interoperability and trust; expected to increase active usage rate.",
        comparable_country="Kenya and Rwanda",
        collected_by="Faysal",
        collection_date="2026-02-01",
        notes="Linked to EVT_0011."
    ),
]

new_rows_df = pd.DataFrame(new_rows)
new_rows_df.head()


,record_id,record_type,category,pillar,indicator,indicator_code,indicator_direction,value_numeric,value_text,value_type,...,impact_direction,impact_magnitude,impact_estimate,lag_months,evidence_basis,comparable_country,collected_by,collection_date,original_text,notes
0,REC_0034,observation,,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,49,,percentage,...,,,,,,,Faysal,2026-02-01,Account ownership 49% in 2024 (Findex 2025),Confirms slow +3pp growth 2021–2024 despite su...
1,REC_0035,observation,,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,57,,percentage,...,,,,,,,Faysal,2026-02-01,Men 57% account ownership (Findex 2025),Male account ownership; persistent gender gap.
2,REC_0036,observation,,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,42,,percentage,...,,,,,,,Faysal,2026-02-01,Women 42% account ownership (Findex 2025),Female account ownership; persistent gender gap.
3,REC_0037,observation,,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,73,,percentage,...,,,,,,,Faysal,2026-02-01,Urban account ownership 73% in 2024,Urban-rural gap remains wide; supports spatial...
4,REC_0038,observation,,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,43,,percentage,...,,,,,,,Faysal,2026-02-01,Rural account ownership 43% in 2024,Rural access lags urban by 30pp.


## 4) Append Enrichments and Validate Schema
Concatenate new rows and run basic sanity checks.

In [6]:
df_enriched = pd.concat([df, new_rows_df], ignore_index=True)

# Ensure column order
missing_cols = [c for c in df.columns if c not in df_enriched.columns]
extra_cols = [c for c in df_enriched.columns if c not in df.columns]
print("Missing cols:", missing_cols)
print("Extra cols:", extra_cols)

df_enriched = df_enriched[df.columns]

print("Enriched shape:", df_enriched.shape)
print("New record_type counts:\n", df_enriched["record_type"].value_counts())

# Quick null check on key fields
key_fields = ["record_id", "record_type", "indicator", "indicator_code", "observation_date"]
print("\nNulls in key fields:\n", df_enriched[key_fields].isna().sum())


Missing cols: []
Extra cols: []
Enriched shape: (57, 34)
New record_type counts:
 record_type
observation    41
event          11
target          3
impact_link     2
Name: count, dtype: int64

Nulls in key fields:
 record_id           0
record_type         0
indicator           0
indicator_code      0
observation_date    0
dtype: int64


## 5) Save Enriched Dataset and Generate Log Entries
Write the enriched CSV to data/processed and generate the enrichment log.

In [8]:
from pathlib import Path

output_dir = root / "data" / "processed"
output_dir.mkdir(parents=True, exist_ok=True)
output_path = output_dir / "ethiopia_fi_enriched_data.csv"

df_enriched.to_csv(output_path, index=False)
print("Saved:", output_path)

log_content = """# Data Enrichment Log

Collection date: 2026-02-01  
Collected by: Faysal

## Additions (Task 1)

- **Account ownership 2024 overall (Findex 2025)**
  - source_url: https://data.worldbank.org/
  - original_text/quote: Account ownership 49% in 2024 (Findex 2025)
  - confidence: high
  - collected_by: Faysal
  - collection_date: 2026-02-01
  - notes: Confirms slow +3pp growth 2021–2024 despite supply boom; baseline for forecasts.

- **Account ownership 2024 men**
  - source_url: https://birrmetrics.com/49-of-ethiopians-are-banked-as-findex-2025-highlights-the-next-inclusion-challenge
  - original_text/quote: Men 57% account ownership (Findex 2025)
  - confidence: high
  - collected_by: Faysal
  - collection_date: 2026-02-01
  - notes: Male account ownership; persistent gender gap.

- **Account ownership 2024 women**
  - source_url: https://birrmetrics.com/49-of-ethiopians-are-banked-as-findex-2025-highlights-the-next-inclusion-challenge
  - original_text/quote: Women 42% account ownership (Findex 2025)
  - confidence: high
  - collected_by: Faysal
  - collection_date: 2026-02-01
  - notes: Female account ownership; persistent gender gap.

- **Account ownership 2024 urban**
  - source_url: https://birrmetrics.com/49-of-ethiopians-are-banked-as-findex-2025-highlights-the-next-inclusion-challenge
  - original_text/quote: Urban account ownership 73% in 2024
  - confidence: high
  - collected_by: Faysal
  - collection_date: 2026-02-01
  - notes: Urban-rural gap remains wide; supports spatial analysis.

- **Account ownership 2024 rural**
  - source_url: https://birrmetrics.com/49-of-ethiopians-are-banked-as-findex-2025-highlights-the-next-inclusion-challenge
  - original_text/quote: Rural account ownership 43% in 2024
  - confidence: high
  - collected_by: Faysal
  - collection_date: 2026-02-01
  - notes: Rural access lags urban by 30pp.

- **Account ownership 2024 richest 60%**
  - source_url: https://birrmetrics.com/49-of-ethiopians-are-banked-as-findex-2025-highlights-the-next-inclusion-challenge
  - original_text/quote: Richest 60% account ownership 53% in 2024
  - confidence: high
  - collected_by: Faysal
  - collection_date: 2026-02-01
  - notes: Income split supports distribution analysis.

- **Account ownership 2024 poorest 40%**
  - source_url: https://birrmetrics.com/49-of-ethiopians-are-banked-as-findex-2025-highlights-the-next-inclusion-challenge
  - original_text/quote: Poorest 40% account ownership 43% in 2024
  - confidence: high
  - collected_by: Faysal
  - collection_date: 2026-02-01
  - notes: Income split supports distribution analysis.

- **Mobile money registered accounts (2025)**
  - source_url: https://birrmetrics.com/ethiopias-mobile-money-users-reach-136-million-digital-transactions-top-96-trillion-birr
  - original_text/quote: Mobile money registered accounts reach 136 million
  - confidence: high
  - collected_by: Faysal
  - collection_date: 2026-02-01
  - notes: Registration surge highlights inactive accounts; leading indicator for usage activation.

- **Telebirr users (late 2025)**
  - source_url: https://stockmarket.et/
  - original_text/quote: Telebirr users reach 58.6 million in late 2025
  - confidence: high
  - collected_by: Faysal
  - collection_date: 2026-02-01
  - notes: Updated user base for late 2025; supports usage growth tracking.

- **CICO agents registered (mid 2025)**
  - source_url: https://nbe.gov.et/wp-content/uploads/2025/07/CICO-Agent-Mapping-Strategy_NBE.pdf
  - original_text/quote: Registered CICO agents about 500000 in mid 2025
  - confidence: high
  - collected_by: Faysal
  - collection_date: 2026-02-01
  - notes: Agent network growth enables rural cash in cash out coverage.

- **Internet penetration (end 2025)**
  - source_url: https://datareportal.com/reports/digital-2026-ethiopia
  - original_text/quote: Internet penetration at 21.7% of population end 2025
  - confidence: high
  - collected_by: Faysal
  - collection_date: 2026-02-01
  - notes: Low but rising connectivity; proxy for app based payments readiness.

- **NDPS 2026–2030 policy event**
  - source_url: https://nbe.gov.et/wp-content/uploads/2025/12/Ethiopia_NDPS_Draft_F.pdf
  - original_text/quote: NDPS 2026-2030 draft launched in Dec 2025
  - confidence: high
  - collected_by: Faysal
  - collection_date: 2026-02-01
  - notes: Policy focus on interoperability and active usage; likely boosts usage pillar.

- **Impact link: NDPS → digital payments**
  - source_url: https://nbe.gov.et/wp-content/uploads/2025/12/Ethiopia_NDPS_Draft_F.pdf
  - original_text/quote: Comparable policy impacts in Kenya and Rwanda; strategy aims to activate dormant accounts.
  - confidence: medium
  - collected_by: Faysal
  - collection_date: 2026-02-01
  - notes: Expected positive impact on digital payment usage within 12–24 months.

- **Impact link: NDPS → active usage rate**
  - source_url: https://nbe.gov.et/wp-content/uploads/2025/12/Ethiopia_NDPS_Draft_F.pdf
  - original_text/quote: Policy focus on interoperability and trust; expected to increase active usage rate.
  - confidence: medium
  - collected_by: Faysal
  - collection_date: 2026-02-01
  - notes: Expected positive impact on usage activation within 18–24 months.
"""

log_path = root / "data_enrichment_log.md"
with open(log_path, "w", encoding="utf-8") as f:
    f.write(log_content)

print("Saved:", log_path)


Saved: D:\10  Academy\week10\Forecasting-Financial-Inclusion-in-Ethiopia\data\processed\ethiopia_fi_enriched_data.csv
Saved: D:\10  Academy\week10\Forecasting-Financial-Inclusion-in-Ethiopia\data_enrichment_log.md
